In [42]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [43]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [44]:
def dominantColor(img):
    data = np.reshape(img, (-1,3))
    data = np.float32(data)
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    compactness,labels,centers = cv2.kmeans(data,1,None,criteria,10,cv2.KMEANS_RANDOM_CENTERS)
    return centers[0].astype(np.int32)

In [45]:
#https://adamspannbauer.github.io/2018/03/02/app-icon-dominant-colors/
from sklearn.cluster import KMeans
from collections import Counter

def get_dominant_color(image, k=4, image_processing_size = None):
    """
    takes an image as input
    returns the dominant color of the image as a list
    
    dominant color is found by running k means on the 
    pixels & returning the centroid of the largest cluster

    processing time is sped up by working with a smaller image; 
    this resizing can be done with the image_processing_size param 
    which takes a tuple of image dims as input

    >>> get_dominant_color(my_image, k=4, image_processing_size = (25, 25))
    [56.2423442, 34.0834233, 70.1234123]
    """
    #resize image if new dims provided
    if image_processing_size is not None:
        image = cv2.resize(image, image_processing_size, 
                            interpolation = cv2.INTER_AREA)
    
    #reshape the image to be a list of pixels
    image = image.reshape((image.shape[0] * image.shape[1], 3))

    #cluster and assign labels to the pixels 
    clt = KMeans(n_clusters = k)
    labels = clt.fit_predict(image)

    #count labels to find most popular
    label_counts = Counter(labels)

    #subset out most popular centroid
    dominant_color = clt.cluster_centers_[label_counts.most_common(1)[0][0]]

    return list(dominant_color)

In [46]:
def biggestRectangleContour(contours):
    biggest = None
    max_area = 0
    indexReturn = -1
    for index in range(len(contours)):
            i = contours[index]
            area = cv2.contourArea(i)
            if area > 100:
                approx = cv2.approxPolyDP(i,0.1*cv2.arcLength(i,True),True)
                if area > max_area and len(approx)==4:
                        biggest = approx
                        max_area = area
                        indexReturn = index
    
    return contours[indexReturn]

In [73]:
def drawContours(path):
    plt.figure(figsize = (20,20))
    
    img = cv2.imread(path)
    #show image
    plt.subplot(1,4,1)
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    #img = cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
    plt.imshow(img)
    
    
#     gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
#     gray = cv2.Canny(gray, 140, 210)
#     contours, hierarchy = cv2.findContours(gray, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    
    #increase contrast always 
    clahe = cv2.createCLAHE()
    gray = clahe.apply(gray)
    
    #show image with increased contrast
    plt.subplot(1,4,2)
    plt.imshow(gray)
    
    invGamma = 1.0 / 0.2
    table = np.array([((i / 255.0) ** invGamma) * 255
    for i in np.arange(0, 256)]).astype("uint8")
    gray = cv2.LUT(gray, table)

    ret,thresh1 = cv2.threshold(gray,130,255,cv2.THRESH_BINARY)
    thresh1 = cv2.erode(thresh1, None, iterations=3)
    thresh1 = cv2.dilate(thresh1, None, iterations=3)
    plt.subplot(1,4,3)
    plt.imshow(thresh1)
    
    # find in white mask
#     lower_white = np.array([0,0,0], dtype=np.uint8)
#     upper_white = np.array([0,0,255], dtype=np.uint8)
#     mask_white = cv2.inRange(img, lower_white, upper_white)

    contours, hierarch = cv2.findContours(thresh1, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    
#     largeContours = []
    
#     #loop contours and bound to a rectangle 
#     for contour in contours:
#         area = cv2.contourArea(contour)
#         if area > 10000:
#             largeContours.append(contour)
            
#     biggestContour = biggestRectangleContour(largeContours)
#     x,y,w,h = cv2.boundingRect(biggestContour)
#     biggestDominantColorVal = get_dominant_color(cv2.cvtColor(img[y:y+h,x:x+w], cv2.COLOR_BGR2HLS),4)
#     print(biggestDominantColorVal)
            
    #loop contours and bound to a rectangle 
    for contour in contours:
        area = cv2.contourArea(contour)
        if area > 90000:
            (x,y,w,h) = cv2.boundingRect(contour)
#             dominantColorVal = get_dominant_color(cv2.cvtColor(img[y:y+h,x:x+w], cv2.COLOR_BGR2HLS),4)
            cv2.rectangle(img, (x,y), (x+w,y+h), (0,255,0), 50)
    
    plt.subplot(1,4,4)
    plt.imshow(img)
    plt.show()
    #cv2.imwrite('./result2_contours.png',output)

In [74]:
drawContours('../input/reddit-img/IMG_8B6F6BBC59FF-1.jpeg')

In [75]:
drawContours('../input/reddit-img/IMG_0018.PNG')

In [69]:
drawContours('../input/reddit-img/IMG_0019.PNG')

In [70]:
drawContours('../input/reddit-img/IMG_0021.PNG')

In [72]:
drawContours('../input/reddit-img/IMG_0024.PNG')
drawContours('../input/reddit-img/IMG_0025.PNG')